## Problem 9: NYPD Motor Vehicle Collisions

In this notebook you will analyze motor collisions in New York City. This data is taken from [NYC Open Data](https://data.cityofnewyork.us/Public-Safety/NYPD-Motor-Vehicle-Collisions/h9gi-nx95). This is a breakdown of every collision in NYC by location and injury. Each record represents a collision in NYC by city, borough, precinct and cross street.

This notebook has a total of 4 exercises worth a total of 10 points.



### Setup
Run the following code cells to setup your environment

Please ensure that **Python 3.5** is selected to run this notebook

In [1]:
import pandas as pd
import numpy as np
import sqlite3 as db

In [2]:
conn = db.connect('../resource/lib/publicdata/mt2nb9/nypd.db')
c = conn.cursor()
c.execute('SELECT type, name, sql FROM sqlite_master')
results = c.fetchall()
for table in results:
    print(table)

('table', 'nypd', 'CREATE TABLE "nypd" (\n"Unique_key" TEXT,\n  "DATE" TEXT,\n  "TIME" TEXT,\n  "BOROUGH" TEXT,\n  "Zip_code" TEXT,\n  "LATITUDE" REAL,\n  "LONGITUDE" REAL,\n  "CONTRIBUTING FACTOR VEHICLE 1" TEXT,\n  "Num_of_persons_injured" INTEGER,\n  "VEHICLE TYPE CODE 1" TEXT\n)')
('table', 'Detail_injured', 'CREATE TABLE "Detail_injured" (\n"Unique_key" TEXT,\n  "Num_of_persons_injured" INTEGER,\n  "Num_of_persons_killed" INTEGER,\n  "Num_of_pedestrians_injured" INTEGER,\n  "Num_of_pedestrians_killed" INTEGER\n)')


### Table Details
Please note that there are two tables in the database

<b>1. Table: nypd</b> 

   <b>Columns:</b> Unique_key, DATE, TIME, BOROUGH, Zip_code, LATITUDE, LONGITUDE, CONTRIBUTING FACTOR VEHICLE 1, Num_of_Persons_injured, VEHICLE TYPE CODE 1
   
   
<b>2. Table: Detail_injured </b>

   <b>Columns:</b> Unique_key, Num_of_persons_injured, Num_of_persons_killed, Num_of_pedestrans_injured, Num_of_pedestrians_killed
   
   Table **`Detail_injured`** is only used in Exercise-3.

### Exercises

**Exercise 0** (2 points) Using the table **`nypd`**, here are some tasks to complete in this exercise.
 
1. Using the column, `DATE`, which is in the form mm/dd/yyyy, add a new column to the dataset called `Month`. 
2. Then return the total number of PERSONS INJURED (total_num_of_injured) in each month, by the descending order of Num_of_injured.
 
 Save your result in a table named **`new_dataset`**.
 
 The format of your table should be:


-----| month | total_num_of_injured |
-----|-------|----------------------|
 0   |   10  |  11234               | 



In [3]:
query='''
select month, sum(Num_of_Persons_injured) as total_num_of_injured
from (
    select substr(DATE,1,2) as month,  Num_of_Persons_injured
    from nypd)
group by month
order by total_num_of_injured desc
'''
new_dataset=pd.read_sql_query(query,conn)
new_dataset.head()

,month,total_num_of_injured
0,07,24136
1,10,23181
2,08,23132
3,09,22977
4,12,22032


In [4]:
## Test Cell: Check Month & Total number of injured ##
assert (new_dataset[new_dataset['month']=='06']['total_num_of_injured'] == 21055.0).all()
assert (new_dataset['total_num_of_injured'].sum() == 245979.0)

print("\n(Passed!)")


(Passed!)


**Exercise 1 (2 point):** Using the table **`nypd`**, here are some tasks to complete in this exercise.
There are many contributing factors for the collisions. please list top 10 contributing factors (unspecified is not a factor). Save your result in a table named **`top10_factors`**, by the descending order of cnt.

The format of your table should be:


-----| CONTRIBUTING FACTOR VEHICLE 1   |  cnt  |
-----|---------------------------------|-------|
 0   |             Lost Consciousness  |  11234| 



In [5]:
###
### YOUR CODE HERE
###

query='''
    select "CONTRIBUTING FACTOR VEHICLE 1", count(*) as cnt 
    from nypd
    where  "CONTRIBUTING FACTOR VEHICLE 1" <> "Unspecified" 
    group by "CONTRIBUTING FACTOR VEHICLE 1"
    order by cnt desc
    limit 10

'''
top10_factors=pd.read_sql_query(query,conn)
print(top10_factors)


    CONTRIBUTING FACTOR VEHICLE 1     cnt
0  Driver Inattention/Distraction  166711
1   Failure to Yield Right-of-Way   58626
2                Backing Unsafely   41497
3                 Other Vehicular   33228
4           Following Too Closely   26373
5                 Fatigued/Drowsy   25211
6              Turning Improperly   23777
7             Passing Too Closely   19249
8  Passing or Lane Usage Improper   18120
9              Lost Consciousness   15152


In [6]:
## Test Cell: Check the contributing factors ##
assert top10_factors['cnt'].sum() == 427944
assert top10_factors['CONTRIBUTING FACTOR VEHICLE 1'].iloc[0] == 'Driver Inattention/Distraction'
assert top10_factors['CONTRIBUTING FACTOR VEHICLE 1'].iloc[9] == 'Lost Consciousness'
assert (top10_factors['CONTRIBUTING FACTOR VEHICLE 1']=='Fatigued/Drowsy').any()

print("\n(Passed!)")


(Passed!)


**Exercise 2 (3 points):** Using the table **`nypd`**, here are some tasks to complete in this exercise.

There are five boroughs in NYC, please list ONE location in QUEENS and ONE location in Bronx that has the `highest` number of collisions.((latitude, longitude) = (0,0) should not be chosen).

Save your result in a table named **`top_collisions`**.
The format of your table should be:


-----| borough  |   latitude | longitude |
-----|----------|------------|-----------|
 0   |  QUEENS  |  40.6807   |-73.8446429|  



In [7]:
query='''
    select *
    from (
    select borough as borough, latitude, longitude, count(*) as collisions
    from nypd 
    where latitude<>0 
      and longitude<>0
      and borough="BRONX"
      group by borough, latitude, longitude
      order by collisions desc
      limit 1)
    union all 
    select * 
    from (
    select borough as borough, latitude, longitude, count(*) as collisions
    from nypd 
    where latitude<>0 
      and longitude<>0
      and borough="QUEENS"
      group by borough, latitude, longitude
      order by collisions desc
      limit 1)
    order by collisions desc
    
'''
top_collisions=pd.read_sql_query(query,conn).drop(["collisions"],axis=1)

# Show your solution:
print(top_collisions)

  borough   latitude  longitude
0  QUEENS  40.733497 -73.870369
1   BRONX  40.820189 -73.890675


In [8]:
## Test Cell: Check the locations ##
assert (top_collisions['borough'].iloc[0]=='QUEENS')
assert (top_collisions['borough'].iloc[1]=='BRONX')
assert (top_collisions['latitude'].sum() == 81.55368630000001)
assert (top_collisions['longitude'].sum() == -147.76104459999999)
assert ((top_collisions['latitude']*top_collisions['longitude']).sum() == -6025.2298195065159)

print("\n(Passed!)")


(Passed!)


### Detail_injured table

Detail_injured table is required for the next question. It has 5 columns, displayed as below.

In terms of Database structures, the `Detail_injured` table's Unique_key column is a primary key, the same as the `NYPD` table's Unique_key column.
This means you can perform any SQL joins or Pandas merges between the two tables on this column.

In [9]:
Detail_injured = pd.read_table('../resource/lib/publicdata/mt2nb9/Detail_injured.txt', sep=',')
print('Number of injured and killed:')
print('-------------------')
print(Detail_injured.head())

Number of injured and killed:
-------------------
   Unique_key  Num_of_persons_injured  Num_of_persons_killed  \
0     4090610                       0                      0   
1     4090611                       0                      0   
2     4090424                       0                      0   
3     4089381                       0                      0   
4     4089728                       0                      0   

   Num_of_pedestrians_injured  Num_of_pedestrians_killed  
0                           0                          0  
1                           0                          0  
2                           0                          0  
3                           0                          0  
4                           0                          0  


**Exercise 3(3 points):** Use table **`nypd`** and table **`detail_injured`**. Both tables have primary key "Unique_key". 

Find the top 10 locations (by Latitude and Longitude) that have the largest number of injured people, which is the sum of `person_injured` and `pedestrians_injured` in table **`detail_injured`**. Please be aware it is not the same as `Num_of_persons_injured` in table **`nypd`**.

Report the locations (by Latitude and Longitude), zip code and the count of collisions for these top 10 locations, by the descending order of num_of_collisions.  

Save your result in a table named **`top_ten_locations`**. ((latitude, longitude) = (0,0) should not be chosen)

The format of your table should be:

-----| LATITUDE | LONGITUDE  | Zip_code |num_of_collisions|
-----|----------|------------|----------|-----------------|
 0   |  40.6807 | -73.8446429|  11200   |    10          |

In [10]:


query='''
    select LATITUDE,LONGITUDE,Zip_code, num_of_collisions from (
    select LATITUDE,LONGITUDE,Zip_code, count(*) as num_of_collisions, sum(injured) as injured
    from (
    select unique_key, latitude, longitude, zip_code
    from nypd 
    where latitude<>-10 
      and longitude<>-10) as A
      inner join 
    (select unique_key, sum(injured) as injured
    from (select unique_key, Num_of_persons_injured as injured
            from Detail_injured
            where Num_of_persons_injured is not NULL
            union all
          select unique_key, Num_of_pedestrians_injured as injured
            from Detail_injured
            where Num_of_pedestrians_injured is not NULL
            ) 
    group by unique_key) as B
    on A.unique_key=B.unique_key
    group by  A.LATITUDE,A.LONGITUDE,A.Zip_code
    order by injured desc
    limit 10
    )
    order by num_of_collisions desc 
'''

top_ten_locations=pd.read_sql_query(query,conn)

print(top_ten_locations)


    LATITUDE  LONGITUDE Zip_code  num_of_collisions
0  40.675736 -73.896853    11207                433
1  40.658578 -73.890623    11207                417
2  40.820189 -73.890675    10459                333
3  40.680015 -73.877428    11208                302
4  40.678163 -73.897477    11207                286
5  40.663183 -73.962449    11225                282
6  40.668798 -73.931120    11213                281
7  40.656160 -73.767353    11434                225
8  40.642058 -74.020420    11220                208
9  40.804376 -73.937420    10035                207


In [11]:
assert top_ten_locations['num_of_collisions'].sum() == 2975 or top_ten_locations['num_of_collisions'].sum() == 2974
assert top_ten_locations[top_ten_locations['Zip_code'] == '11207']['num_of_collisions'].sum() == 1136
#assert top_ten_locations.iloc[6]['LATITUDE'] == 40.66879779999999
assert top_ten_locations.iloc[6]['LONGITUDE'] == -73.93112009999999  
#assert top_ten_locations.iloc[3]['LONGITUDE'] == -73.8774282
#assert top_ten_locations.iloc[3]['LATITUDE'] == 40.680015000000004
#assert top_ten_locations.iloc[7]['Zip_code'] == '11434'

In [12]:
## Test Cell: Check the locations ##
assert top_ten_locations['num_of_collisions'].sum() == 2975 or top_ten_locations['num_of_collisions'].sum() == 2974
assert top_ten_locations[top_ten_locations['Zip_code'] == '11207']['num_of_collisions'].sum() == 1136
assert top_ten_locations.iloc[6]['LATITUDE'] == 40.66879779999999
assert top_ten_locations.iloc[6]['LONGITUDE'] == -73.93112009999999  
assert top_ten_locations.iloc[3]['LONGITUDE'] == -73.8774282
assert top_ten_locations.iloc[3]['LATITUDE'] == 40.680015000000004
assert top_ten_locations.iloc[7]['Zip_code'] == '11434'

print("\n(Passed!)")


(Passed!)


In [13]:
# Some cleanup code
conn.close()

** Note: ** You've reached the end of this problem. Don't forget to restart the kernel and run the entire notebook from top-to-bottom to make sure you did everything correctly. If that is working, try submitting this problem. (Recall that you *must* submit and pass the autograder to get credit for your work.)